In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
df = pd.read_csv("dvf.csv", sep=",", encoding="utf-8")

C:\Users\darkj\AppData\Local\Temp\ipykernel_45332\2401041495.py:1: DtypeWarning: Columns (10,12,14,16,17,18,20,22,24,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dvf.csv", sep=",", encoding="utf-8")


In [3]:
print(df.columns)

Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'ancien_code_commune',
       'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle',
       'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude'],
      dtype='object')


In [4]:
df = df[df['valeur_fonciere'].notnull()]

In [5]:
df.to_csv('dvf_clean.csv', index=False)

In [2]:
df_bordeaux = df[df['code_departement'] == 33]
# print(df_bordeaux)
df_bordeaux.to_csv('Bordeaux.csv', index=False)

NameError: name 'df' is not defined

In [ ]:
# Renommer pour homogénéiser (optionnel)
df_map = df_bordeaux.rename(columns={"latitude": "lat", "longitude": "long"}).copy()

# Filtrer les coordonnées non nulles
df_map = df_map.dropna(subset=["lat", "long"])

# Si les coordonnées sont avec des virgules au lieu de points (cas rare)
df_map["lat"] = df_map["lat"].astype(str).str.replace(",", ".").astype(float)
df_map["long"] = df_map["long"].astype(str).str.replace(",", ".").astype(float)

# Garder uniquement les points en France métropolitaine
df_map = df_map[(df_map["lat"] >= 41) & (df_map["lat"] <= 51) &
                (df_map["long"] >= -5) & (df_map["long"] <= 10)]

# Créer la carte centrée sur la France
m = folium.Map(location=[46.6, 2.2], zoom_start=6)

# Ajouter un cluster de marqueurs
marker_cluster = MarkerCluster().add_to(m)

# Ajouter chaque point au cluster
for _, row in df_map.iterrows():
    popup = f"""
    <b>Commune:</b> {row['nom_commune']}<br>
    <b>Adresse:</b> {row['adresse_numero']} {row['adresse_nom_voie']}<br>
    <b>Valeur foncière:</b> {row['valeur_fonciere']} €<br>
    <b>Date:</b> {row['date_mutation']}
    """
    folium.Marker(
        location=[row["lat"], row["long"]],
        popup=popup
    ).add_to(marker_cluster)

# Afficher la carte dans Jupyter
m


In [1]:
# Liste des colonnes numériques et catégorielles
colonnes_numeriques = df_bordeaux.select_dtypes(include=["float", "int"]).columns.tolist()
colonnes_categoriels = df_bordeaux.select_dtypes(include=["object"]).columns.tolist()

# Widgets de sélection
x_dropdown = widgets.Dropdown(
    options=df.columns,
    description="Axe X:"
)

y_dropdown = widgets.Dropdown(
    options=colonnes_numeriques,
    description="Axe Y:"
)

graph_type = widgets.RadioButtons(
    options=["Histogramme", "Boxplot", "Nuage de points"],
    description="Type:"
)

# Fonction de mise à jour du graphique
def update_plot(x_col, y_col, chart_type):
    clear_output(wait=True)
    display(x_dropdown, y_dropdown, graph_type)
    plt.figure(figsize=(10, 5))
    
    if chart_type == "Histogramme":
        sns.histplot(data=df, x=x_col, bins=30)
    elif chart_type == "Boxplot":
        sns.boxplot(data=df, x=x_col, y=y_col)
    elif chart_type == "Nuage de points":
        sns.scatterplot(data=df, x=x_col, y=y_col)
    
    plt.xticks(rotation=45)
    plt.title(f"{chart_type} de {y_col} selon {x_col}")
    plt.tight_layout()
    plt.show()

# Liaison des widgets
widgets.interactive(update_plot, x_col=x_dropdown, y_col=y_dropdown, chart_type=graph_type)

NameError: name 'df_bordeaux' is not defined

In [ ]:
# df_paris = df[df['code_departement'] == 75]
# # print(df_paris)
# df_paris.to_csv('Paris.csv', index=False)

In [ ]:
# df_toulouse = df[df['code_departement'] == 31]
# # print(df_toulouse)
# df_toulouse.to_csv('Toulouse.csv', index=False)

In [ ]:
# df_nantes = df[df['code_commune'] == 44109]
# # print(df_nantes)
# df_nantes.to_csv('Nantes.csv', index=False)

In [ ]:
# df_angouleme = df[df['code_commune'] == 16015]
# # print(df_angouleme)
# df_angouleme.to_csv('Angouleme.csv', index=False)